## Prepare the Data

### Load library

기본적인 라이브러리를 로드한다.
이번 베이스라인은 케라스 pretrained 모델 기반으로 실행된다.

In [ ]:
import gc
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

from keras import backend as K
warnings.filterwarnings(action='ignore')

K.image_data_format()

### File load

주어진 파일을 확인하고 로드한다.

In [ ]:
DATA_PATH = '../input'
os.listdir(DATA_PATH)

> os.path.join : 경로를 병합하여 새 경로를 생성.

ex) os.path.join('C:\Tmp', 'a', 'b') : "C:\Tmp\a\b"

In [ ]:
# 이미지 폴더 경로
TRAIN_IMG_PATH = os.path.join(DATA_PATH, 'train')
TEST_IMG_PATH = os.path.join(DATA_PATH, 'test')

# CSV 파일 경로
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
df_class = pd.read_csv(os.path.join(DATA_PATH, 'class.csv'))

## Data Exploration

실제 데이터가 Description과 일치하는지, 데이터는 어떻게 구성되어 있고 클래스 별로 어떤 분포를 가지고 있는지 등 데이터에 대한 전반적인 궁금증을 해결해보는 과정.

### Check Data

Data Description에 나와 있는 컬럼 별 세부 설명.

- img_file : 데이터셋의 각 로우와 연결되는 이미지 파일 이름.
- bbox_x1 : 바운딩 박스 x1 좌표 (좌상단 x)
- bbox_y1 : 바운딩 박스 y1 좌표 (좌상단 y)
- bbox_x2 : 바운딩 박스 x2 좌표 (우하단 x)
- bbox_y2 : 바운딩 박스 y2 좌표 (우하단 y)
- class : 예측하려는 차종(Target)
- id : 각 데이터셋에 기입되어 있는 클래스 id
- name : 클래스 id에 대응되는 실제 차종 레이블

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
# Data 누락 체크
if set(list(df_train.img_file)) == set(os.listdir(TRAIN_IMG_PATH)):
    print("Train file 누락 없음!")
else:
    print("Train file 누락")
    
if set(list(df_test.img_file)) == set(os.listdir(TEST_IMG_PATH)):
    print("Test file 누락 없음!")
else:
    print("Test file 누락")

In [ ]:
# Data 개수
print("Number of Train Data : {}".format(df_train.shape[0]))
print("Number of Test Data : {}".format(df_test.shape[0]))

In [ ]:
df_class.head()

In [ ]:
print("타겟 클래스 총 개수 : {}".format(df_class.shape[0]))
print("Train Data의 타겟 종류 개수 : {}".format(df_train['class'].nunique()))

### Class Distribution

분류 문제에서 가장 먼저 의심해봐야 할 부분이 바로 Target Class의 분포입니다. 학습에 사용해야 하는 Train Set의 타겟 분포를 확인해서 밸런스가 어느 정도인지 체크해야 합니다.

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(df_train["class"], order=df_train["class"].value_counts(ascending=True).index)
plt.title("Number of data per each class")
plt.show()

In [ ]:
cntEachClass = df_train["class"].value_counts(ascending=False)
print("Class with most count : {}".format(cntEachClass.index[0]))
print("Most Count : {}".format(cntEachClass.max()))

print("Class with fewest count : {}".format(cntEachClass.index[-1]))
print("Fewest Count : {}".format(cntEachClass.min()))

print("Mean : {}".format(cntEachClass.mean()))

In [ ]:
cntEachClass.describe()

In [ ]:
cntEachClass

### Image Visualization

파이썬 커널에서 이미지를 로드하는 방법은 여러가지가 있지만, 이 커널에서는 PIL 라이브러리를 사용합니다.

In [ ]:
import PIL
from PIL import ImageDraw

tmp_imgs = df_train['img_file'][100:110]
plt.figure(figsize=(12, 20))

for num, f_name in enumerate(tmp_imgs):
    img = PIL.Image.open(os.path.join(TRAIN_IMG_PATH, f_name))
    plt.subplot(5, 2, num+1)
    plt.title(f_name)
    plt.imshow(img)
    plt.axis('off')

### Bounding Box

> 바운딩 박스란?

이미지 내부에서 특정 Object를 박스로 레이블한 좌표를 말하며, 보통 좌측 상단 (x1, y1)과 우측 하단(x2, y2) 좌표가 주어져서 직사각형 모양의 박스를 그릴 수 있게 됩니다. 이 때, 좌표는 이미지의 픽셀 좌표입니다.

In [ ]:
def draw_rect(drawcontext, pos, outline=None, width=0):
    (x1, y1) = (pos[0], pos[1])
    (x2, y2) = (pos[2], pos[3])
    points = (x1, y1), (x2, y1), (x2, y2), (x1, y2), (x1, y1)
    drawcontext.line(points, fill=outline, width=width)
    
def make_boxing_img(img_name):
    if img_name.split('_')[0] == "train":
        PATH = TRAIN_IMG_PATH
        data = df_train
    elif img_name.split('_')[0] == "test":
        PATH = TEST_IMG_PATH
        data = df_test
        
    img = PIL.Image.open(os.path.join(PATH, img_name))
    pos = data.loc[data["img_file"] == img_name, ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2']].values.reshape(-1)
    draw = ImageDraw.Draw(img)
    draw_rect(draw, pos, outline='red', width=10)
    
    return img

In [ ]:
f_name = "train_00102.jpg"

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)

# Original Image
origin_img = PIL.Image.open(os.path.join(TRAIN_IMG_PATH, f_name))
plt.title("Original Image - {}".format(f_name))
plt.imshow(origin_img)
plt.axis('off')

# Image included bounding box
plt.subplot(1, 2, 2)
boxing = make_boxing_img(f_name)
plt.title("Boxing Image - {}".format(f_name))
plt.imshow(boxing)
plt.axis('off')

plt.show()

왼쪽 그림과 같이 어떤 이미지에는 내가 필요로 하는 Target Object 뿐만 아니라 상관 없는 다른 Object(Noise)가 섞여 있을 수 있습니다. 이런 경우에 이미지 내부에서 필요한 Object를 명확히 표시하기 위해 Bounding Box를 사용합니다. (실제로 이미지를 모델에 넣을 때는 Box 바깥 부분은 잘라서 사용합니다.)

이번 컴페티션은 Bounding Box 좌표가 이미 주어져 있습니다. 만약 Bounding Box 좌표가 주어지지 않는다면 직접 레이블을 하거나, 좌표를 예측하는 딥러닝 모델을 설계해볼 수도 있습니다.

## Model

이번 커널에서는 ResNet50 Pretrained Model을 불러와서 사용합니다.

### Train set, Valid set Split

모델 학습을 하기 전에 주어진 Train 데이터셋을 어떻게 활용할 지 생각해야합니다.

#### Train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

df_train["class"] = df_train["class"].astype('str')

df_train = df_train[['img_file', 'class']]
df_test = df_test[['img_file']]

its = np.arange(df_train.shape[0])
train_idx, val_idx = train_test_split(its, train_size = 0.8, random_state = 42)

X_train = df_train.iloc[train_idx, :]
X_val = df_train.iloc[val_idx, :]

print(X_train.shape)
print(X_val.shape)
print(df_test.shape)

### Generator

> Generator의 이점?

제너레이터는 코랩이나 캐글 커널같은 클라우드 환경 또는 일반적인 로컬 환경에서 정말 유용하게 쓰일 수 있습니다. 그 이유는 보통 이러한 환경은 메모리가 충분하지 않기 때문입니다. 특히나 이미지처럼 파일 하나의 용량이 매우 큰 경우, 한 번에 모든 파일을 메모리에 적재하게 되면 상당히 큰 부담이 됩니다. 배치사이즈 단위만큼 파일을 불러와 학습하고 끝나면 다시 불러와서 학습하는 방법을 반복하기 때문에 전체 학습을 하더라도 메모리를 조금만 사용하게 되는 것입니다.

> Keras DataGenerator

지금껏 불편하게 제너레이터를 만들어 사용했다면 케라스에는 정말 편한 제너레이터 함수가 있씁니다.
케라스 ImageDataGenerator는 제너레이터의 기능은 물론 제너레이터를 정의하면서 동시에 Data에 원하는 Noise까지 부여할 수 있습니다.

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator

# Parameter
img_size = (224, 224)
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
nb_test_samples = len(df_test)
epochs = 20
batch_size = 32

# Define Generator config
train_datagen = ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = False,
    zoom_range = 0.10,
    preprocessing_function = preprocess_input)

val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

# Make Generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe = X_train,
    directory = '../input/train',
    x_col = 'img_file',
    y_col = 'class',
    target_size = img_size,
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = batch_size,
    seed = 42
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=X_val, 
    directory='../input/train',
    x_col = 'img_file',
    y_col = 'class',
    target_size = img_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe = df_test,
    directory = '../input/test',
    x_col = 'img_file',
    y_col = None,
    target_size = img_size,
    color_mode = 'rgb',
    class_mode = None,
    batch_size = batch_size,
    shuffle = False
)

### Loading Pretrained Model - ResNet50

보통 딥러닝 모델을 구성할 때 직접 만들어 보는것도 좋지만, 이 작업은 상당히 많은 시간과 노력이 필요하기 때문에 이미 성능이 입증된 모델을 불러와서 사용해보는 것도 좋은 방법입니다.

Pretrained Model을 불러오기 위해서는 커널의 Internet 옵션이 활성화 되어 있어야 합니다.

In [ ]:
resNet_model = ResNet50(include_top = False, input_shape = (224, 224, 3))
# resNet_model.summary()

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, GlobalAveragePooling2D

model = Sequential()
model.add(resNet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(196, activation='softmax', kernel_initializer='he_normal'))
model.summary()

Pretrained Model을 사용할 때 한 가지 주의할 점이 있습니다.

Pretrained 모델은 경우에 따라 다양하게 사용될 수 있기 때문에 Model output 부분을 잘라버린 채 로드되는 경우가 있습니다. (include_top = False)

이 경우에는 직접 output을 만들어야하므로 우리는 196개의 class를 분류하기 때문에 위와 같이 만들었습니다.

참고)
Keras에는 모델을 생성하는 방법이 2가지가 있습니다.

하나는 위 처럼 Sequential을 사용하는 것이고 다른 하나는 Model을 사용하는 방법입니다.

2가지 모두 많이 사용하니 Model도 한 번 사용해보시길 권유드립니다.

### Model Compile

이제 Model을 만들었으니 어떻게 학습할 지 정해야합니다. 어떤 방법으로, 어떤 속도로, 어떤 지표를 기준으로 등등 정할 수 있고 필요시에는 각각의 함수를 직접 구현해볼 수도 있습니다. 하지만 보통은 기본으로 주어지는 것들을 사용합니다.

In [ ]:
from sklearn.metrics import f1_score

def micro_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average='micro')

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

### Model Training

이제 진짜로 학습을 시작해봅시다!

In [ ]:
def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0:
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size

In [ ]:
%%time
from keras.callbacks import ModelCheckpoint, EarlyStopping

filepath = "my_resnet_model_{val_acc:.2f}_{val_loss:.4f}.h5"
es = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')

callbackList = [es]

history = model.fit_generator(
    train_generator,
    steps_per_epoch = get_steps(nb_train_samples, batch_size),
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = get_steps(nb_validation_samples, batch_size),
    callbacks = callbackList
)

gc.collect()

### Training History Visualization

학습된 결과를 plot으로 그려볼 수 있습니다. 모델 학습 로그를 통해서 확인할 수도 있지만, 전반적인 학습 형태를 한 눈에 파악하기에는 그래프만 한 것이 없습니다.

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Predict & Make submission

모델이 테스트 데이터에도 잘 적용되는지 predict를 해보고 제출물을 만들어봅시다.

### Model Predict

In [ ]:
%%time
test_generator.reset()
prediction = model.predict_generator(
    generator = test_generator,
    steps = get_steps(nb_test_samples, batch_size),
    verbose = 1
)

Predict 또한 제너레이터를 사용합니다. 제너레이터는 메모리가 부족한 우리들에게 꼭 필요한 기능입니다!

### Make Submission

Inference가 끝난 결과를 이제 sample_submission 파일에 매핑해야 합니다.

sample_submission 파일을 불러온 후 예측한 결과를 매핑합니다.

**중요**
케라스 제너레이터를 사용하는 경우에는 타겟(클래스)의 카테고리컬 매핑이 제너레이터 임의로 결정됩니다. 따라서 제너레이터가 가지고 있는 class index 딕셔너리를 불러와 새롭게 매핑해주어야 합니다.

In [ ]:
predicted_class_indices=np.argmax(prediction, axis=1)

# Generator class dictionary mapping
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

submission = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))
submission["class"] = predictions
submission.to_csv("submission.csv", index=False)
submission.head()